In [291]:
import numpy as np
import pandas as pd


lex = pd.read_csv('Lexique380.utf8.csv')
lex = lex[(lex.ortho.str.contains('-| ') == False) & (lex.phon.str.contains('°') == False)]  # suppress schwa
lex = lex[(lex.ortho.str.contains('-') == False)]
lex = lex[['ortho', 'p_cvcv']]
dic = lex.set_index('ortho').to_dict()
dic = dic['p_cvcv']

In [324]:
def get_LPC_g(sentence):
    sentence = sentence.lower()
    sentence = sentence.replace('.', '')
    snt_to_wlst = sentence.split(" ")
    LPC_gustures = 0 
    e = 0
    no_in_db = []
    
    for word in snt_to_wlst:
        if word in dic:
            #print(word, dic[word])
            e = 1
            if dic[word][0] == 'V':
                LPC_gustures += 1
            for letter in dic[word]:
                if letter == 'C':
                    LPC_gustures += 1
        else: 
            e = 0
        if e == 0 :
            no_in_db.append(word)
    
    if no_in_db == []:
        return f'{LPC_gustures}'
    else:
        return f'{LPC_gustures}, except {no_in_db}'


    


In [325]:
print(get_LPC_g("Les faux jumeaux ֳ©taient aussi diffֳ©rents que le jour et la nuit."))

13, except ['ֳ©taient', 'diffֳ©rents']


## Counting the numbers of LPC gestures for both sentences in to the file

In [326]:
df = pd.read_csv('test1.csv')

for index,row in df.iterrows():
    p_sentence = row['"Predictable end" sentence']
    pred_end_n_LPC = get_LPC_g(p_sentence)
    df.loc[index,"pred_end_n_LPC"] = pred_end_n_LPC
    unp_sentence = row['"Unpredictable end" sentence']
    unpred_end_n_LPC = get_LPC_g(unp_sentence)
    df.loc[index,"unpred_end_n_LPC"] = unpred_end_n_LPC

df.to_csv('CUSTIME_sentence_predictability_with_LPC.csv')



